In [ ]:
from pathlib import Path
from dataclasses import dataclass
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
from casacore.tables import table, taql

from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u




In [ ]:

ASKAP = EarthLocation.of_site("ASKAP")

@dataclass
class Baselines:
    ant_xyz: np.ndarray
    b_xyz: np.ndarray
    b_idx: np.ndarray
    b_map: dict[tuple[int,int], int]

@dataclass
class HourAngle:
    hour_angle: u.rad
    time_mjds: np.ndarray
    location : EarthLocation
    position: SkyCoord
    elevation: np.ndarray
    time: Time
    time_map: dict[float, int]


def make_baseline_vector(ms_path: Path) -> Baselines:
    """Cakculcate the baseline vectors from the antenna table"""
    with table(str(ms_path / "ANTENNA")) as tab:
        ants_idx = np.arange(len(tab), dtype=int)
        b_idx = np.array(
            list(combinations(list(ants_idx), 2))
        )
        xyz = tab.getcol("POSITION")
        b_xyz = (xyz[b_idx[:, 0]] - xyz[b_idx[:, 1]])
    
    b_map = {tuple(k): idx for idx, k in enumerate(b_idx)}
    
    print(f"ants={len(ants_idx)}, baselines={b_idx.shape[0]}")
    return Baselines(
        ant_xyz=xyz*u.m, b_xyz=b_xyz*u.m, b_idx=b_idx, b_map=b_map
    )

def make_hour_angles(
    ms_path: Path, 
    obs_location: EarthLocation = ASKAP, 
    whole_day: bool = False,
    add_lon_offset: bool = False,
) -> HourAngle:
    """Work out the hour angle and assoicated time-dependent components"""
    with table(str(ms_path / "FIELD")) as tab:
        field_positions = tab.getcol("PHASE_DIR")
        position = SkyCoord(field_positions[0]*u.rad)

    with table(str(ms_path)) as tab:
        times_mjds = tab.getcol("TIME")
        
        # sorts time stemps on uniquness and order they first appear
        times_mjds, indicies = np.unique(times_mjds, return_index=True)
        sorted_idx = np.argsort(indicies)
        times_mjds = times_mjds[sorted_idx]
        
        delta = 0
        times_mjds =  times_mjds - delta
        time_map = {k: idx for idx, k in enumerate(times_mjds)}

    if whole_day:
        time_step = times_mjds[1] - times_mjds[0]
        times_mjds = times_mjds[0] + time_step * np.arange(int(60*60*24/time_step))
    
    times = Time(times_mjds/60/60/24, format="mjd", scale="utc", location=EarthLocation.of_site("greenwich"))
    
    
    lst = times.sidereal_time("apparent", longitude=obs_location)
    hour_angle = position.ra - lst
    
    if add_lon_offset:
        """Shift the hour angles by the longitude of the observatory. 
        
        Why? I have no clue.
        """
        hour_angle = hour_angle - obs_location.lon
    

    alt = np.arcsin(
        np.sin(obs_location.lat.rad) * 
        np.sin(position[0].dec.rad) +
        np.cos(obs_location.lat.rad) *
        np.cos(position.dec.rad) *
        np.cos(hour_angle)
    )

    return HourAngle(
        hour_angle=hour_angle, 
        time_mjds=times_mjds, 
        location=obs_location, 
        position=position,
        elevation=alt.to(u.deg),
        time=times,
        time_map=time_map
    )




In [ ]:

ms_path = Path("scienceData.EMU_1141-55.SB47138.EMU_1141-55.beam00_averaged_cal.leakage.ms")

baselines = make_baseline_vector(ms_path=ms_path)
hour_angles = make_hour_angles(ms_path=ms_path)

# !!!!!!!!!!!!!!!!!!!!!!!!!
# Why is this extra obsrvatory shift in longitude needed?
# !!!!!!!!!!!!!!!!!!!!!!!!!
local_askap_hour_angles = make_hour_angles(ms_path=ms_path, add_lon_offset=True)
# !!!!!!!!!!!!!!!!!!!!!!!!!



In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot(
    hour_angles.time_mjds,
    hour_angles.elevation,
    label="tim",
    color="black"
)


ax.plot(
    local_askap_hour_angles.time_mjds,
    local_askap_hour_angles.elevation,
    label="tim w. ASKAP lon offset",
    color="red"
)


ax.set(
    xlabel="MJD (seconds)",
    ylabel="Elevation (deg)",
    title="Elevation to phase direction"
)
ax.legend()
ax.grid()



In [ ]:
def get_uvws_from_ms(ms_path: Path, ant_1: int, ant_2: int) -> np.ndarray:
    with table(str(ms_path), ack=False) as tab:
        with taql("SELECT * FROM $tab WHERE ANTENNA1=$ant_1 AND ANTENNA2=$ant_2") as sub:
            uvws = sub.getcol("UVW")

    return uvws        
        
def all_b_xyz_to_uvw(baselines: Baselines, hour_angle: HourAngle):
    ha = hour_angle.hour_angle
    ha = ha.to(u.rad)
    
    b_xyz = baselines.b_xyz

    declination = hour_angle.position.dec.to(u.rad)
    declination = (np.ones(len(ha)) * declination).decompose()
    
    sin_ha = np.sin(ha)
    sin_dec = np.sin(declination)
    cos_ha = np.cos(ha)
    cos_dec = np.cos(declination)
    zeros = np.zeros_like(sin_ha)
    mat = np.array([
        [sin_ha, cos_ha, zeros],
        [-sin_dec * cos_ha, sin_dec*sin_ha, cos_dec],
        [cos_dec*cos_ha, -cos_dec * sin_ha, sin_dec]
    ])
    
    uvw = np.einsum('ijk,lj->lki', mat, b_xyz,  optimize=True)
    print(f"{uvw.shape=}")
    return uvw

In [ ]:
tim_uvws = all_b_xyz_to_uvw(baselines=baselines, hour_angle=hour_angles)
tim_local_askap_uvws = all_b_xyz_to_uvw(baselines=baselines, hour_angle=local_askap_hour_angles)

# Get the antena_1=0 and antenna_2=1 baseline
tim_bl_uvws = tim_uvws[0]
tim_local_askap_bl_uvws = tim_local_askap_uvws[0]

ms_bl_uvws = get_uvws_from_ms(ms_path=ms_path, ant_1=0, ant_2=1)

print(f"{tim_uvws.shape=}")
print(f"{ms_bl_uvws.shape=}")


fig, ax = plt.subplots(1,1)

ax.scatter(
    ms_bl_uvws[:, 0], ms_bl_uvws[:, 1], label="ms"
)

ax.scatter(
    tim_bl_uvws[:, 0], tim_bl_uvws[:, 1], label="tim", color="black"
)

ax.scatter(
    tim_local_askap_bl_uvws[:, 0], 
    tim_local_askap_bl_uvws[:, 1], 
    label="tim w. ASKAP longitude offset",
    marker="x",
    color="red"
)


ax.set(
    xlabel="u (meters)",
    ylabel="v (meters)",
    title="(u,v)s for baseline (0, 1)"
)
ax.legend()
ax.grid()




In [ ]:
with table(str(ms_path)) as tab:
    print(tab.colnames())
    print(tab.getcol("TIME"))
    print(tab.getcol("TIME_CENTROID"))